In [1]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [3]:
# import csv
data = pd.read_csv("mbti_1.csv")
data.columns = ['type', 'posts']

In [4]:
import string
#calculating the average post length
data['avg_post_len'] = data['posts'].apply(lambda x: (len(x) - x.count(" "))/50)
data.head()

,type,posts,avg_post_len
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,81.50
1,ENTP,'I'm finding the lack of me in these posts ver...,117.20
2,INTP,'Good one _____ https://www.youtube.com/wat...,87.94
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",103.42
4,ENTJ,'You're fired.|||That's another silly misconce...,102.50


In [5]:
#calculating the total post length
data['tot_post_len'] = data['posts'].apply(lambda x: len(x) - x.count(" "))
data.head()

,type,posts,avg_post_len,tot_post_len
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,81.50,4075
1,ENTP,'I'm finding the lack of me in these posts ver...,117.20,5860
2,INTP,'Good one _____ https://www.youtube.com/wat...,87.94,4397
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",103.42,5171
4,ENTJ,'You're fired.|||That's another silly misconce...,102.50,5125


In [6]:
# calculating the punctuation percentage
def punct_count(post):
    count = sum([1 for char in post if char in string.punctuation])
    return round(count/(len(post) - post.count(" ")), 3)*100

data['punct_%'] = data['posts'].apply(lambda x: punct_count(x))
data.head()

,type,posts,avg_post_len,tot_post_len,punct_%
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,81.50,4075,12.7
1,ENTP,'I'm finding the lack of me in these posts ver...,117.20,5860,9.0
2,INTP,'Good one _____ https://www.youtube.com/wat...,87.94,4397,9.5
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",103.42,5171,8.9
4,ENTJ,'You're fired.|||That's another silly misconce...,102.50,5125,8.6


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [8]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
stopwords =nltk.corpus.stopwords.words('english')
from nltk.stem import PorterStemmer
ps = nltk.PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
#nltk.download()

In [10]:
# create function to clean the posts
def clean_posts(post):
    post = "".join([word.lower()for word in post if word not in string.punctuation])
    tokens = re.split('\W+', post)
    post = [ps.stem(word) for word in tokens if word not in stopwords]
    return post

#TF-IDF
tf_vectorize = TfidfVectorizer(analyzer=clean_posts)
X_tf = tf_vectorize.fit_transform(data['posts'])
X_tf_feature = pd.concat([data['avg_post_len'], data['punct_%'], pd.DataFrame(X_tf.toarray())], axis=1)

#Count Vectorizer
count_vectorize = CountVectorizer(analyzer = clean_posts)
X_count = count_vectorize.fit_transform(data['posts'])
X_count_feature = pd.concat([data['avg_post_len'], data['punct_%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feature.head()

MemoryError: Unable to allocate 20.2 GiB for an array with shape (8675, 312793) and data type float64